In [ ]:
!pip install flask flask-ngrok pyngrok transformers accelerate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.9 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken "your authtoken"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
#from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import google.generativeai as genai
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

In [ ]:
from huggingface_hub import login

login("your token")

In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = "your api"

In [ ]:
# Load Qwen2-1.5B-Instruct
def load_qwen_model():
    model_id = "Qwen/Qwen2-1.5B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
        trust_remote_code=True
    )
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load Qwen3-0.6B
def load_model2():
    model_id = "Qwen/Qwen3-0.6B"
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
        trust_remote_code=True
    )
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load Gemma-2B-IT
def load_gemma_model():
    model_id = "google/gemma-2b-it"
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
        trust_remote_code=True
    )
    return pipeline("text-generation", model=model, tokenizer=tokenizer)


# Load Yi-1.5-9B-Chat (4-bit quantized)
def load_yi_model_and_tokenizer(model_id: str = "01-ai/Yi-1.5-9B-Chat"):
    nf4_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=nf4_config,
            device_map="auto",
            trust_remote_code=True
        )
        tokenizer = AutoTokenizer.from_pretrained(
            model_id,
            trust_remote_code=True
        )
        return pipeline("text-generation", model=model, tokenizer=tokenizer)
    except Exception as e:
        print(f"Error loading Yi model: {e}")
        return None

# Load Gemini 2.0 client
def load_gemini_model_client(model_name: str = 'gemini-2.0-flash-lite'):
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        print("Gemini API key not found.")
        return None
    try:
        genai.configure(api_key=api_key)
        return genai.GenerativeModel(model_name)
    except Exception as e:
        print(f"Gemini load error: {e}")
        return None

# Load all models at startup
print("🔄 Loading models...")
qwen2_pipeline = load_qwen_model()
qwen3_pipeline = load_model2()
gemma_pipeline = load_gemma_model()
yi_pipeline = load_yi_model_and_tokenizer()
gemini_model = load_gemini_model_client()
print("✅ All models loaded.")

🔄 Loading models...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

Device set to use cuda:0


✅ All models loaded.


In [ ]:
# Setup Flask
app = Flask(__name__)
# run_with_ngrok(app)  # Expose to public via ngrok

In [ ]:
import json
import re

def extract_json_from_text(text):


    try:
        # Extract the first JSON object using regex (non-greedy)
        match = re.search(r'\{[\s\S]*?\}', text)
        if not match:
            print("❌ No JSON found in output")
            return None

        json_str = match.group(0)

        # Attempt to parse it
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"Initial JSON decode error: {e}")

            # Fix common issues
            cleaned = (
                json_str.replace('\n', ' ')
                        .replace('\r', ' ')
                        .replace("“", '"')
                        .replace("”", '"')
                        .replace("‘", "'")
                        .replace("’", "'")
                        .replace("\\'", "'")
                        .replace('\\"', '"')
                        .strip()
            )
            return json.loads(cleaned)

    except Exception as e:
        print(f"JSON extraction error: {e}")
        return None


In [ ]:
# Define root route
@app.route('/')
def index():
    return "Welcome! Use the `/summarize` endpoint with a POST request containing 'text' and 'model'."

@app.route('/summarize', methods=['POST'])
def summarize():
    data = request.get_json()
    text = data.get("text")
    model_name = data.get("model")

    if not text or not model_name:
        return jsonify({"error": "Missing 'text' or 'model'"}), 400

    prompt = (

     "Summarize the following text precisely into the sections:\n"
    "- Date of Execution\n"
    "- Vendor details\n"
    "- Vendee details\n"
    "- Nature of Deed\n"
    "- Consideration Amount\n"
    "- Property Description\n"
    "- Property Location\n"
    "- Property Boundaries\n"
    "Return the result strictly as a valid JSON object with these keys and no additional commentary.\n"
    f"Text to summarize:\n{text}\n\n"
    "JSON:"



    )

    try:
        if model_name == "qwen2":
            result = qwen2_pipeline(prompt, max_new_tokens=1000, temperature=0.2, return_full_text=False)[0]["generated_text"]

        elif model_name == "qwen3":
            result = qwen3_pipeline(prompt, max_new_tokens=1000, temperature=0.2, return_full_text=False)[0]["generated_text"]

        elif model_name == "gemma":
            result = gemma_pipeline(prompt, max_new_tokens=1000, temperature=0.2, return_full_text=False)[0]["generated_text"]

        elif model_name == "yi":
            result = yi_pipeline(prompt, max_new_tokens=1000, temperature=0.2, return_full_text=False)[0]["generated_text"]

        elif model_name == "gemini":
            if gemini_model:
                response = gemini_model.generate_content(prompt)
                result = response.text
            else:
                return jsonify({"error": "Gemini model not initialized."}), 500

        else:
            return jsonify({"error": f"Unsupported model: {model_name}"}), 400

        # Try direct JSON parsing
        try:
            json_data = json.loads(result.strip())
        except json.JSONDecodeError:
            # Fallback to regex-based JSON extraction
            json_data = extract_json_from_text(result)
        if json_data is None:

            return jsonify({"model": model_name, "summary": result}), 200

        return jsonify({
            "model": model_name,
            "summary": json_data
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(" * ngrok tunnel available at:", public_url)

# Run the Flask app
app.run(port=5000)

 * ngrok tunnel available at: NgrokTunnel: "https://496d-34-83-12-133.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/May/2025 07:39:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/May/2025 07:39:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/May/2025 07:40:27] "POST /summarize HTTP/1.1" 200 -


Initial JSON decode error: Expecting ',' delimiter: line 14 column 4 (char 1557)
JSON extraction error: Expecting ',' delimiter: line 1 column 1558 (char 1557)


INFO:werkzeug:127.0.0.1 - - [30/May/2025 07:44:17] "POST /summarize HTTP/1.1" 200 -


Initial JSON decode error: Expecting ',' delimiter: line 1 column 240 (char 239)
JSON extraction error: Expecting ',' delimiter: line 1 column 240 (char 239)
